<a href="https://colab.research.google.com/github/ruebot/notebooks/blob/main/yfile_indigenous_mallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cribbed from https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/10-Topic-Modeling-CSV.html

In [ ]:
%%capture

import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version   
install_java()

!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [ ]:
%%capture

!pip install little_mallet_wrapper
!pip install seaborn
!pip install git+https://github.com/maria-antoniak/little-mallet-wrapper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/maria-antoniak/little-mallet-wrapper.git to /tmp/pip-req-build-lh6q73nz
  Running command git clone -q https://github.com/maria-antoniak/little-mallet-wrapper.git /tmp/pip-req-build-lh6q73nz


In [ ]:
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path
import pandas as pd
import random
pd.options.display.max_colwidth = 100

os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
path_to_mallet = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
data = "https://ruebot.net/indigenous-web-pages.csv"

In [ ]:
web_pages = pd.read_csv(data)

In [ ]:
web_pages

,crawl_date,domain,url,mime_type_web_server,mime_type_tika,language,content
0,20220617202108,yorku.ca,https://yfile.news.yorku.ca/2012/05/08/innovations-in-undergraduate-education-focus-of-event-on-...,text/html,text/html,en,2007-10_seifert – YFile Skip to content Awards & Recognition Brainstorm Editor's Picks Features ...
1,20211101125012,yorku.ca,https://yfile.news.yorku.ca/2021/09/29/a-statement-from-the-indigenous-council-on-this-day-for-t...,text/html,text/html,en,A statement from the Indigenous Council on this day for Truth and Reconciliation – YFile Skip to...
2,20211115175513,yorku.ca,https://yfile.news.yorku.ca/2020/02/05/two-cutting-edge-projects-on-ai-and-human-interaction-awa...,text/html,text/html,en,AIFEATURED – YFile Skip to content Awards & Recognition Brainstorm Editor's Picks Features Innov...
3,20211117211229,yorku.ca,https://yfile.news.yorku.ca/2020/09/24/faculty-of-liberal-arts-professional-studies-expands-with...,text/html,text/html,en,Abigail Shabtay – YFile Skip to content Awards & Recognition Brainstorm Editor's Picks Features ...
4,20220415191046,yorku.ca,https://yfile.news.yorku.ca/2021/11/30/announcement-of-the-renewal-of-provost-and-vice-president...,text/html,text/html,fr,Announcement of the renewal of provost and vice-president academic – YFile Skip to content Award...
...,...,...,...,...,...,...,...
1227,20220517221939,yorku.ca,https://yfile.news.yorku.ca/2022/03/29/new-book-club-brings-non-indigenous-and-indigenous-people...,text/html,text/html,en,book-cover-a-mind-spread-out-on-the-ground-by-alicia-elliott – YFile Skip to content Awards & Re...
1228,20221015222809,yorku.ca,https://yfile.news.yorku.ca/2022/10/05/york-students-explore-dark-matter-in-germany/group-photo-...,text/html,text/html,en,group photo in frankfurt copy FEATURED image for YFile – YFile Skip to content Aspire Awards & R...
1229,20211116004831,yorku.ca,https://yfile.news.yorku.ca/2021/11/02/mclaughlin-college-serves-up-full-menu-for-november-lunch...,text/html,text/html,en,image-2 – YFile Skip to content Awards & Recognition Brainstorm Editor's Picks Features Innovatu...
1230,20220518060742,yorku.ca,https://yfile.news.yorku.ca/2019/12/04/a-road-map-back-home-jesse-thistle-and-the-power-of-story...,text/html,text/html,en,kitty lundy panelists – YFile Skip to content Awards & Recognition Brainstorm Editor's Picks Fea...


In [ ]:
training_data = [little_mallet_wrapper.process_string(text, numbers='remove') for text in web_pages['content']]

In [ ]:
original_texts = [text for text in web_pages['content']]

In [ ]:
little_mallet_wrapper.print_dataset_stats(training_data)

Number of Documents: 1232
Mean Number of Words per Document: 375.7
Vocabulary Size: 27101


In [ ]:
num_topics = 15

In [ ]:
training_data = training_data

In [ ]:
output_directory_path = 'topic-modeling'
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"

In [ ]:
little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
                                             output_directory_path,
                                             num_topics,
                                             training_data)

Importing data...
Complete
Training topic model...
Complete


([['york',
   'university',
   'students',
   'research',
   'faculty',
   'education',
   'community',
   'program',
   'learning',
   'new',
   'school',
   'teaching',
   'student',
   'support',
   'professor',
   'president',
   'indigenous',
   'work',
   'development',
   'academic'],
  ['york',
   'university',
   'team',
   'lions',
   'year',
   'first',
   'top',
   'two',
   'student',
   'canadian',
   'women',
   'back',
   'weekend',
   'four',
   'three',
   'season',
   'soccer',
   'men',
   'player',
   'game'],
  ['indigenous',
   'aboriginal',
   'law',
   'osgoode',
   'first',
   'canada',
   'peoples',
   'day',
   'nations',
   'school',
   'hall',
   'national',
   'rights',
   'legal',
   'nation',
   'centre',
   'women',
   'york',
   'professor',
   'justice'],
  ['york',
   'subscribe',
   'yfile',
   'newsletter',
   'news',
   'share',
   'contact',
   'campus',
   'latest',
   'editor',
   'scoop',
   'archive',
   'picks',
   'awards',
   'media',
   

In [ ]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")


✨Topic 0✨

['york', 'university', 'students', 'research', 'faculty', 'education', 'community', 'program', 'learning', 'new', 'school', 'teaching', 'student', 'support', 'professor', 'president', 'indigenous', 'work', 'development', 'academic']

✨Topic 1✨

['york', 'university', 'team', 'lions', 'year', 'first', 'top', 'two', 'student', 'canadian', 'women', 'back', 'weekend', 'four', 'three', 'season', 'soccer', 'men', 'player', 'game']

✨Topic 2✨

['indigenous', 'aboriginal', 'law', 'osgoode', 'first', 'canada', 'peoples', 'day', 'nations', 'school', 'hall', 'national', 'rights', 'legal', 'nation', 'centre', 'women', 'york', 'professor', 'justice']

✨Topic 3✨

['york', 'subscribe', 'yfile', 'newsletter', 'news', 'share', 'contact', 'campus', 'latest', 'editor', 'scoop', 'archive', 'picks', 'awards', 'media', 'explores', 'jewish', 'research', 'recognition', 'home']

✨Topic 4✨

['york', 'top', 'university', 'stories', 'campus', 'news', 'contact', 'yfile', 'twitter', 'current', 'day', 'co